# Exercise 07: Radiotelescope

\- Gugliemo Grillo

<div><table>
  <tr>
    <td style="width: 45%"><img src="https://i.ibb.co/5nGRFtm/Radiowave-test2020-1.png"/></td>
    <td style="width: 45%"><img src="https://i.ibb.co/C5wZ6K6/Radiowave-test2020-2.png"/></td>
  </tr>
  <tr><td style="width: 45%"><img src="https://i.ibb.co/2jp0Vdm/Radiowave-test2020-3.png"/></tr></td>
    
</table></div>

#### 1.1-2 PSD
The plot on page 1 provides the temperature required to evaluate the PSD. The output PSD can be obtained by multiplying by the square of the low pass transfer function.
$|h(f)|^2 = \frac{1}{1+(f/f_0)^2}$

$$S_{V,V}^{sky}(\omega=2\pi f) = \frac{\gamma}{1+(f/f_0)^2} (T_{sky}+T_{rec})(2\pi f_{lo}) ^2$$

$$S_{V,V}^{cold}(\omega=2\pi f) = \frac{\gamma}{1+(f/f_0)^2} (T_{R}+T_{rec})(2\pi f_{lo})^2$$

Where $\gamma=\frac{4 k_b}{3 \epsilon_0 c^3}$. The temperature is read by eye from the graph.

In [3]:
# packages used
import scipy.constants
from scipy.integrate import quad

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display, Math

# Given constans
kb = scipy.constants.Boltzmann
Epsilon0 = scipy.constants.epsilon_0
c = scipy.constants.c

f0 = 10e6 #Hz
Tn = { # I'll use the value found by the professor
    '700MHz':{'sky': 16.0, 'rec':51.0, 'Rcold': 2},
    '30MHz': {'sky': 2.8, 'rec': 38.0, 'Rcold': 2},
     '2GHz': {'sky': 8.1, 'rec': 140.0,'Rcold': 2}
}

# Parameters redefinition
gamma = 4*kb / (3*Epsilon0*c**3) # prefactor

def PSD(flo, T, f):
    return gamma*T*np.power(2*np.pi*flo, 2)/(1+ np.power(f/f0, 2) )

display(Math(r'\gamma = {:.5e}\ [s^2\ V^2 / (m^2\ Hz\ K)]'.format(gamma)))

<IPython.core.display.Math object>

The numerator of the output PSD is the PSD that arrives at the low pass and is equal to:

In [9]:
# As we set f=0 we are evaluating only the numerator of the transfer function
PSD_TOT =  PSD(700e6, Tn['700MHz']['sky'],  f=0) + PSD(700e6, Tn['700MHz']['rec'], f=0)
PSD_Rcold= PSD(700e6, Tn['700MHz']['Rcold'],f=0) + PSD(700e6, Tn['700MHz']['rec'], f=0)

display(Math(r'\text{Numerator: }'+'700Mhz: {:.5e}\ [V^2 / (m^2\ Hz)]'.format(PSD_TOT)))
display(Math(r'\text{Numerator } R_{cold}:'+ '{:.5e}'.format(PSD_Rcold)+'\ [V^2 / (m^2\ Hz)]'))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

#### 2. Temperature of the noise coming from the antenna.

<!--
The temperature of noise from the antenna can be evaluated by inverting the given relationship:
$$(S_{V,V}^{sky} - S_{V,V}^{cold})(f) = \frac{\gamma}{1+(f/f_0)^2} (T_{sky}-T_{R})(2\pi f_{lo}) ^2 \longrightarrow
T_{sky} =\frac{1+(f/f_0)^2}{\gamma (2\pi f_{lo})^2 } (S_{V,V}^{sky} - S_{V,V}^{cold})(f) + T_{R}$$ -->


As the output is squared and integrated over a time t we are evaluating the variance of the process. This can be extrapolated from the PSD as

$$<E^2_{sky/R}> = R(0) = \frac{1}{2\pi}\int_{-\infty}^{+\infty}S(\omega)d\omega = \gamma (T_{sky/R} + T_{rec}) (2\pi f_{lo})^2 \pi f_0$$

where $\pi f_0$ is the result of the integral of the transfer function. It's then possible to invert this relation to get:

$$T_{sky} = \frac{<E^2_{sky}>-<E^2_{R}>}{\gamma (2\pi f_{lo})^2 \pi f_0} + T_{R}$$


The resolution on the individual measurements is given by the radiometric formula:

$$\frac{\sigma_{E^2}}{E^2} = \frac{1}{\sqrt{t f_0}} \longrightarrow \sigma_{E^2} = \frac{E^2}{\sqrt{t f_0}} =  \frac{\gamma T (2\pi f_{lo})^2 \pi f_0}{\sqrt{t f_0}}$$

By combining the two errors it's possible to obtain the total error as a function of $\sigma_{sky/R}$ which, in turn, depend on $t$:

$$\sigma_{T_{sky}} = \frac{ \sqrt{\sigma ^2_{sky}+\sigma^2 _r} }{\gamma \pi f_0 (2\pi f_{lo})^2}$$

To get a $5%$ relative resolution one needs to impose $\frac{\sigma_{T_{sky}}}{T_{sky}(f_{lo})} = 0.5$

In [ ]:
# Here I'll make the calculations

#Results: 0.001, 0.016, 0.025

#### 4. Minimum achivable error

This is a Wiener filter problem. As the carrier of the signal is at a frequency much higher than the cut off frequency ($f_s >> f_0$) the sinusoidal will be filtered by the low pass. The net signal is therefore:

$$E_0 e^{- \frac{(t-t_0)^2}{2 \Delta t^2}} \rightarrow \mathcal{FT} \rightarrow E_0 \frac{e^{-\Delta t \omega ^2 /2}\sqrt{2\pi}\Delta t}{1+i 2\pi f/f_0}$$

The minimum error can be evaluated by assuming with noise and a perpetual signal:

$$ \sigma ^2 = \frac{1}{\frac{1}{2\pi} \int_{-\infty}^{+\infty} SNR(\omega) dw}$$

with

$$ SNR(\omega) = \frac{\frac{e^{-\Delta t \omega ^2 /2}\sqrt{2\pi}\Delta t}{1+i 2\pi f/f_0}}{S_{V,V}(\omega=2\pi f_s)}$$

The temperature used are $T_{sky}=2.9K$ and $T_{rec} = 37K$.

#### 5. Power transmitted

The power can be evaluated with the formula provided: $P = I A =\frac{1}{2} \epsilon_0 c A [E(t)]^2 $ while the FT can be asked to Wolfram Alpha.